# Introduction to DriveNet

DriveNet is a DNN able to drive an agent along a specific route. Among others, DriveNet can do lane-following while respecting traffic lights.

### Imitation learning and its limitations
DriveNet is trained using imitation learning on our Lyft L5 Prediction Dataset 2020. We feed examples of real driving experiences to the model and expect it to take the same actions as the driver did in those episodes. This is very similar to how tasks like classification are usually solved.

Imitiation Learning is powerful, but it has a strong limitation. It's not trivial for a trained model to generalise well on out-of-distribution data.

After training DriveNet, we would like it to take full control and drive the AV in an autoregressive fashion (i.e. by following its own predictions).

During evaluation it's very easy for errors to compound and make the AV drift away from the original distribution. In fact, during training our DriveNet has seen good examples of driving only. In particular, this means **almost perfect midlane following**. However, even a small constant displacement during evaluate can accumulate enough error to lead the AV completely out of its distribution in a matter of seconds.

![drifting](../../images/drivenet/drifting.svg)

In this notebook you're going to train your own Drivenet model using the Lyft L5 Prediction Dataset and L5Kit. You will use a technique named **online trajectory perturbation** to mitigate the effects of drifting.

**Before starting, please download the [Lyft L5 Prediction Dataset 2020](https://self-driving.lyft.com/level5/prediction/) and follow [the instructions](https://github.com/lyft/l5kit#download-the-datasets) to correctly organise it.**


In [ ]:
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.geometry import transform_points
from l5kit.visualization import TARGET_POINTS_COLOR, draw_trajectory
from l5kit.drivenet.model import DriveNetModel
from l5kit.kinematic import AckermanPerturbation
from l5kit.random import GaussianRandomGenerator

import os

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/tmp/l5kit_data"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./drivenet_config.yaml")

# Adding perturbations to the mix

One of the simpler techniques to ensure a good generalisation is **data augmentation**, which exposes the network to different versions of the input and helps it to generalise better to out-of-distribution situations.

In our setting, we want to ensure **our DriveNet can recover if it ends up slightly off the midlane it is following**.

To this end, we can enrich the training set with **online trajectory perturbations**. These perturbations are kinematically feasible and affect both starting angle and position. A new ground truth trajectory is then generated to link this new starting point with the original trajectory end point. These starting point will be slightly rotated and off the original midlane, and the new trajectory will teach the model how to recover from this situation.

![perturbation](../../images/drivenet/perturb.svg)


In the following cell, we load the training data and leverage L5Kit to add these perturbations to our training set.
We also plot the same example with and without perturbation. During training, our model will see also those examples and learn how to recover from positional and angular offsets.

In [ ]:
perturb_prob = cfg["train_data_loader"]["perturb_probability"]

# rasterisation and perturbation
rasterizer = build_rasterizer(cfg, dm)
perturbation = AckermanPerturbation(
        random_offset_generator=GaussianRandomGenerator(mean=np.array([0.0, 0.0]), std=np.array([1.0, np.pi / 6])),
        perturb_prob=perturb_prob,
    )

# ===== INIT DATASET
train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()
train_dataset = EgoDataset(cfg, train_zarr, rasterizer, perturbation)

# plot same example with and without perturbation
for perturbation_value in [1, 0]:
    perturbation.perturb_prob = perturbation_value

    data_ego = train_dataset[0]
    im_ego = rasterizer.to_rgb(data_ego["image"].transpose(1, 2, 0))
    target_positions = transform_points(data_ego["target_positions"], data_ego["raster_from_agent"])
    draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)
    plt.imshow(im_ego[::-1])
    plt.axis('off')
    plt.show()

# before leaving, ensure perturb_prob is correct
perturbation.perturb_prob = perturb_prob

## Model
L5Kit provides a model file for DriveNet. The backbone is a ResNetX (either 18 or 50) model pre-trained on ImageNet. Here, we describe the main inputs and outputs of the model; for a full description please check [the class definition](https://github.com/lyft/l5kit/blob/1d077fdbc8656057516bee2bb6cc815bc6868d29/l5kit/l5kit/drivenet/model.py#L9).

#### Inputs
L5Kit is shipped with various rasterisers. They all capture the information around the AV and project it into a fixed grid of pixels we can feed to our CNN backbone. Each rasteriser has its own input representation and, in general, it's not an RGB image. As an example, the ego bounding box and agents are stored in additional channels in the semantic rasteriser.


#### Outputs

During train, the loss value is computed and returned, while the full outputs are returned during the evaluation phase. These are the future positional and angular offsets.

![model](../../images/drivenet/model.svg)


In [ ]:
model = DriveNetModel(
        model_arch="resnet50",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
        criterion=nn.MSELoss(reduction="none")
        )
print(model)

# Prepare for training
Our `EgoDataset` inherits from PyTorch `Dataset`; so we can use it inside a `Dataloader` to enable multi-processing

In [ ]:
train_cfg = cfg["train_data_loader"]
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print(train_dataset)

# Training loop
Here, we purposely include a barebone training loop. Clearly, many more components can be added to enrich logging and improve performance. Still, the sheer size of our dataset ensures that a reasonable performance can be obtained even with this simple loop.

In [ ]:
tr_it = iter(train_dataloader)
progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
losses_train = []
model.train()
torch.set_grad_enabled(True)

for _ in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    # Forward pass
    result = model(data)
    loss = result["loss"]
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses_train.append(loss.item())
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

### Plot the train loss curve
We can plot the train loss against the iterations (batch-wise) to check if our model has converged.

In [ ]:
plt.plot(np.arange(len(losses_train)), losses_train, label="train loss")
plt.legend()
plt.show()

# Store the model

Let's store the model as a torchscript. This format allows us to re-load the model and weights without requiring the class definition later.

**Take note of the path, you will use it later to evaluate your own DriveNet model!**

In [ ]:
to_save = torch.jit.script(model)
path_to_save = f"{gettempdir()}/drivenet.pt"
to_save.save(path_to_save)
print(f"MODEL STORED at {path_to_save}")

# Congratulations in training your first DriveNet model!
### What's Next

Now that your model is trained and safely stored, you can evaluate how it performs in different situations.

First, you can check if it can produce sensible predictions in the **open loop** setting using the dedicated notebook (TODO LINK). 

Then, you can jump to our second evaluation notebook (TODO LINK) to check how it would perform as the actual driver of the AV in what we call the **close loop** setting.